In [1]:
from ib_insync import * 
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop()

In [2]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [3]:
ib.positions()

[]

In [8]:
dt.datetime.utcnow().time()

datetime.time(16, 42, 9, 64424)

In [5]:
freq = "1 min"
sma = 20
dev = 1
units = 1000
end_time = dt.time(16, 25, 0) # stop condition
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [6]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["SMA"] = df["close"].rolling(sma).mean()
        df["Lower"] = df["SMA"] - df["close"].rolling(sma).std() * dev
        df["Upper"] = df["SMA"] + df["close"].rolling(sma).std() * dev
        df["distance"] = df["close"] - df.SMA
        df["position"] = np.where(df["close"] < df.Lower, 1, np.nan)
        df["position"] = np.where(df["close"] > df.Upper, -1, df["position"])
        df["position"] = np.where(df.distance * df.distance.shift(1) < 0, 0, df["position"])
        df["position"] = df.position.ffill().fillna(0)
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

In [7]:
# start trading session
session_start = pd.to_datetime(dt.datetime.utcnow()).tz_localize("utc")
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
last_bar = bars[-1].date
bars.updateEvent += onBarUpdate

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if dt.datetime.utcnow().time() >= end_time: # if stop conditions has been met
        execute_trade(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass

,close,SMA,Lower,Upper,distance,position
date,,,,,,
2024-04-08 21:15:00+00:00,1.085820,NaN,NaN,NaN,NaN,0.0
2024-04-08 21:16:00+00:00,1.085820,NaN,NaN,NaN,NaN,0.0
2024-04-08 21:17:00+00:00,1.085815,NaN,NaN,NaN,NaN,0.0
2024-04-08 21:18:00+00:00,1.085830,NaN,NaN,NaN,NaN,0.0
2024-04-08 21:19:00+00:00,1.085815,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...
2024-04-09 16:21:00+00:00,1.085885,1.085786,1.085672,1.085901,0.000099,-1.0
2024-04-09 16:22:00+00:00,1.085835,1.085796,1.085687,1.085906,0.000039,-1.0
2024-04-09 16:23:00+00:00,1.085720,1.085801,1.085698,1.085904,-0.000081,0.0


,side,cumQty,avgPrice,realizedPNL,cumPNL
time,,,,,
2024-04-09 16:20:02+00:00,SLD,1000.0,1.08585,0.00,0.00
2024-04-09 16:24:01+00:00,BOT,1000.0,1.08575,-3.90,-3.90
2024-04-09 16:25:01+00:00,BOT,1000.0,1.08575,0.00,-3.90
2024-04-09 16:26:57+00:00,SLD,1000.0,1.08570,-4.05,-7.95


Session Stopped.
